In [ ]:
# Test that it works somewhat
import h5py
from matplotlib.axes import Axes
from wildfire.data_types import *
from wildfire.data_utils import *
from wildfire.training_utils import *
from matplotlib.axes import Axes
import matplotlib.pyplot as plt

def plot_metrics(ax: Axes, run_dir: str, key: str, label: str, color: str):
    """Plot validation metrics from multiple training runs.
    
    Args:
        ax: Matplotlib axes to plot on
        run_dir: Base directory containing the run folders
        key: Metric key to plot from validation_metrics.json
        label: Label prefix for the legend
        color: Base color for the plots
    """
    paths = sorted(glob(f"{run_dir}_*/validation_metrics.json"))
    files = [json_load(p) for p in paths]

    # Convert color string to RGBA
    runs = []
    for i, by_epoch in enumerate(files):
        vals = [m[key] for m in by_epoch]
        runs.append(vals)
    mean = np.mean(runs, axis=0)
    std = np.std(runs, axis=0)
    x = np.arange(len(mean))
    plt.plot(x, mean, label=label, color=color)
    plt.fill_between(x, mean - std, mean + std, color=color, alpha=0.2)  # Std shading



fig, ax = plt.subplots()
# eval on modis
modis_dir = "/proj/cvl/users/x_juska/data/wildfire/runs/97" # main, 0
viirs_dir = "/proj/cvl/users/x_juska/data/wildfire/runs/104" # 1
# eval on viirs
# modis_dir = "/proj/cvl/users/x_juska/data/wildfire/runs/64" # 0
# viirs_dir = "/proj/cvl/users/x_juska/data/wildfire/runs/51" # main, 1

plot_metrics(ax, modis_dir, "0.5/iou", "MOD14", "red")
plot_metrics(ax, viirs_dir, "0.5/iou", "VNP14", "blue")
ax.set_xlabel("Epoch")
ax.set_ylabel("IoU [%]")
ax.set_title("Validation IoU across 5 runs")
ax.legend()
out_path = os.path.join(config.root_path, "figures", "validation_iou.pdf")
fig.savefig(out_path, dpi=150)
plt.show()
print(out_path)
